In [1]:
# write your code from here
import pandas as pd
import great_expectations as ge

# Load your dataset
# For demonstration, let's create an example DataFrame here:
data = {
    "customer_id": [1, 2, 3, 4],
    "purchase_amount": [100.0, 250.5, 75.25, 300.0],
    "age": [25, 40, 58, 30]
}
df = pd.DataFrame(data)

# Convert to a Great Expectations DataFrame
ge_df = ge.from_pandas(df)

# Task 1: Validate Column Existence
result_column_exists = ge_df.expect_column_to_exist('customer_id')

# Task 2: Validate Column Data Types
# Great Expectations doesn't have a direct dtype checker on columns, but we can check values are floats.
# We check if all values in 'purchase_amount' are numbers and specifically floats.
result_purchase_amount_type = ge_df.expect_column_values_to_be_of_type('purchase_amount', 'float64')

# Task 3: Validate Range of Values
result_age_range = ge_df.expect_column_values_to_be_between('age', min_value=18, max_value=65)

# Compile and print results
print("Task 1: Column 'customer_id' existence check:", result_column_exists)
print("Task 2: Column 'purchase_amount' data type check (float64):", result_purchase_amount_type)
print("Task 3: Column 'age' range check (18-65):", result_age_range)



AttributeError: module 'great_expectations' has no attribute 'from_pandas'